In [ ]:
!pip install ultralytics kaggle tqdm albumentations

In [ ]:
!git clone https://github.com/n24q02m/VehicleDetection.git
%cd VehicleDetection

In [ ]:
from utils.kaggle_auth import setup_kaggle_auth

# Set up Kaggle authentication
if not setup_kaggle_auth():
    raise Exception("Failed to set up Kaggle authentication")

In [ ]:
from utils.download_utils import download_dataset, download_model

# Download required files
download_dataset()
download_model()

In [ ]:
!python ./src/training/train.py

In [ ]:
from utils.update import update_model

# Update model on Kaggle
update_model(
    model_name="n24q02m/final-vehicle-detection-model",
    model_dir="./runs/final-model",
    title="Final Vehicle Detection Model",
)